## Performance evaluation
... or let's be Morningstar for a minute

We will evaluate just under 2,000 active mutual funds relative to a market model.  The returns data are already in excess of the risk-free rate.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go

In [2]:
# mf = pd.read_csv("../data/mf_rets.csv")
mf = pd.read_csv("https://www.dropbox.com/s/e1cuahqqdn56l0y/mf_rets.csv?dl=1")

In [3]:
mf.columns

Index(['Date', 'MKT-RF', 'RF', '1', '2', '3', '4', '5', '6', '7',
       ...
       '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912',
       '1913'],
      dtype='object', length=1916)

In [4]:
cols = mf.columns

In [5]:
cols = cols[3:]

In [6]:
cols

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912',
       '1913'],
      dtype='object', length=1913)

## Fund Performance: Market Model Alpha

In [7]:
y = mf['1']
X = sm.add_constant(mf['MKT-RF'])
results = sm.OLS(y,X, missing='drop').fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      1   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     2804.
Date:                Mon, 10 Apr 2023   Prob (F-statistic):          2.84e-128
Time:                        11:09:30   Log-Likelihood:                 676.10
No. Observations:                 225   AIC:                            -1348.
Df Residuals:                     223   BIC:                            -1341.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0009      0.001     -1.151      0.2

In [8]:
results.params

const    -0.000933
MKT-RF    0.924485
dtype: float64

In [9]:
df = pd.DataFrame(dtype=float,columns=['alpha','beta'], index=cols)
X = sm.add_constant(mf['MKT-RF'])
for c in cols:
    y = mf[c]
    results = sm.OLS(y,X, missing='drop').fit()
    df.loc[c,'alpha'] = results.params[0]
    df.loc[c,'beta'] = results.params[1]

In [11]:
df

,alpha,beta
1,-0.000933,0.924485
2,0.000034,1.066703
3,-0.001191,1.005108
4,-0.001104,0.729851
5,-0.002545,1.264373
...,...,...
1909,-0.001335,0.997558
1910,0.000863,1.099683
1911,-0.000391,0.509993
1912,0.000417,1.000028


In [12]:
frac_pos = (df.alpha >0).mean()
print(frac_pos)

0.41766858337689494


In [13]:
# Plot the distribution of alphas
fig = go.Figure()
trace= go.Histogram(x=df.alpha, histnorm='percent',hovertemplate="<br>%{y:.2}% of funds<br><extra></extra>")
fig.add_trace(trace)
# some formatting
fig.update_traces(marker_line_width=1, marker_line_color='black')
fig.layout.xaxis["title"] = "Monthly Alpha"
fig.layout.yaxis["title"] = "Percent of Funds"
fig.add_vline(x=0, line_width=4, line_dash="dash", line_color="black")
fig.add_annotation(x=df.alpha.max()*0.7, y=8,
            text="Fraction of Funds with Positive Alpha <br>"+f'{frac_pos:.1%} of funds', showarrow=False)
fig.show()

## Predicting Survival

In [14]:
last = mf.iloc[-1]
last_obs = pd.DataFrame(last[cols])
last_obs.columns=['last']
df = df.merge(last_obs,left_index=True, right_index=True)

In [15]:
df['died'] = np.isnan(df['last'])*1.0

In [20]:
df

,alpha,beta,last,died
1,-0.000933,0.924485,0.0276,0.0
2,0.000034,1.066703,0.0585,0.0
3,-0.001191,1.005108,0.0512,0.0
4,-0.001104,0.729851,NaN,1.0
5,-0.002545,1.264373,NaN,1.0
...,...,...,...,...
1909,-0.001335,0.997558,0.0283,0.0
1910,0.000863,1.099683,0.0662,0.0
1911,-0.000391,0.509993,0.0195,0.0
1912,0.000417,1.000028,0.0434,0.0


In [17]:
# Use alpha to predict leaving sample
y = df['died']
X = sm.add_constant(df['alpha'])
results = sm.OLS(y,X, missing='drop').fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   died   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     210.9
Date:                Mon, 10 Apr 2023   Prob (F-statistic):           2.10e-45
Time:                        11:12:35   Log-Likelihood:                -1273.5
No. Observations:                1913   AIC:                             2551.
Df Residuals:                    1911   BIC:                             2562.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4143      0.011     38.031      0.0

In [18]:
# Let's standardize the alpha first
y = df['died']
X = sm.add_constant((df.alpha - df.alpha.mean())/df.alpha.std())
results = sm.OLS(y,X, missing='drop').fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   died   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     190.5
Date:                Mon, 10 Apr 2023   Prob (F-statistic):           2.23e-41
Time:                        11:12:58   Log-Likelihood:                -1273.5
No. Observations:                1913   AIC:                             2551.
Df Residuals:                    1911   BIC:                             2562.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4381      0.011     40.647      0.0

#### How should we interpret the above result?

## Survivorship bias

In [23]:
df = df[df['died']==0]
df.describe()

,alpha,beta,last,died
count,1075.000000,1075.000000,1075.000000,1075.0
mean,0.000360,1.038410,0.046399,0.0
std,0.002086,0.156203,0.015075,0.0
min,-0.010812,-0.022087,-0.012500,0.0
25%,-0.000973,0.946906,0.034750,0.0
50%,0.000064,1.025742,0.046400,0.0
75%,0.001559,1.125275,0.057550,0.0
max,0.010883,2.157380,0.098600,0.0


In [24]:
frac_pos = (df.alpha >0).mean()
print(frac_pos)

0.5237209302325582


In [25]:
# Plot the distribution of alphas WITH SURVIVORSHIP BIAS
fig = go.Figure()
trace= go.Histogram(x=df.alpha, histnorm='percent',hovertemplate="<br>%{y:.2}% of funds<br><extra></extra>")
fig.add_trace(trace)
# some formatting
fig.update_traces(marker_line_width=1, marker_line_color='black')
fig.layout.xaxis["title"] = "Monthly Alpha"
fig.layout.yaxis["title"] = "Percent of Funds"
fig.add_vline(x=0, line_width=4, line_dash="dash", line_color="black")
fig.add_annotation(x=df.alpha.max()*0.7, y=8,
            text="Fraction of Funds with Positive Alpha <br>"+f'{frac_pos:.1%} of funds', showarrow=False)
fig.show()